In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
df = pd.read_csv(r"data/sessions.csv", index_col=0)

C:\Users\mirzo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
def delete_data(df: pd.DataFrame):
    df = df.drop(['client_id', 'session_id', 'utm_keyword', 'device_os', 'device_model', 'device_brand'], axis=1)
    return df.dropna()

In [46]:
def to_types(df: pd.DataFrame):
    df['visit_datetime'] = pd.to_datetime(df.apply(lambda x: x["visit_date"] + ' ' + x["visit_time"], axis=1))
    df = df.drop(['visit_date', 'visit_time'], axis=1)
    return df

In [30]:
def preprocess(df: pd.DataFrame):
    df = delete_data(df)
    df = to_types(df)
    return df

In [49]:
df = preprocess(df)